# Data Loader and Preparation for USE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# you can obtain the model from https://www.kaggle.com/models/google/universal-sentence-encoder/tensorFlow1/lite
# pick the Tensorflow1 lite version mode

use_model_path = '/content/drive/MyDrive/ndev-task-tracker/universal-sentence-encoder-tensorflow1-lite-v2'


use_model = hub.load(use_model_path)
sp = spm.SentencePieceProcessor()
sp.load(f"{use_model_path}/assets/universal_encoder_8k_spm.model")

# Preprocess & embedding for USE

In [ ]:
# USE Lite is designed to be smaller and mobile/web-friendly, so it does not contain its own tokenizer.
# because the embed_fn signature of the Universal Sentence Encoder Lite model you loaded expects the input in this sparse format. If you tried to pass a dense tensor or just a list of token IDs directly, the model would likely throw an error.

def to_sparse(sentences):
    # Encode sentences to list of token ids
    ids = [sp.encode(s) for s in sentences]

    # Create values and indices for SparseTensor
    values = [token for sent in ids for token in sent]
    indices = [[i, j] for i, sent in enumerate(ids) for j in range(len(sent))]
    dense_shape = [len(ids), max(len(sent) for sent in ids)]

    # Convert to required tensors
    return {
        "values": tf.constant(values, dtype=tf.int64),
        "indices": tf.constant(indices, dtype=tf.int64),
        "dense_shape": tf.constant(dense_shape, dtype=tf.int64),
    }


def embed(sentences):
  embed_fn = use_model.signatures["default"]
  sparse_input = to_sparse(sentences)
  embeddings = embed_fn(**sparse_input)['default']
  return embeddings.numpy()